In [1]:
import re
import pandas as pd
import spacy
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

In [2]:
# Загрузка файла Excel из сессионного хранилища Colab
file_path = '/content/sample_data/ks_vac23.xlsx'
data_for = pd.read_excel(file_path)

data_for.iloc[:, 0] = data_for.iloc[:, 0].astype(str).str.lower()
data_for


,raw_skills
0,"figma, ui, английский язык, ux, android, ios"
1,"английский язык, amocrm, телефония, программир..."
2,"пользователь пк, adobe photoshop, грамотная ре..."
3,"деловая коммуникация, грамотность, поиск инфор..."
4,"linux, postgresql, sql, ms sql, java, oracle, ..."
...,...
10484,"кибербезопасность, информационная безопасность"
10485,асу тп
10486,"администрирование сетевого оборудования, настр..."
10487,"linux, ci/cd pipeline, k8s, okd, ocp, менторст..."


In [3]:
# Подсчет количества уникальных строк
unique_rows_count = data_for.drop_duplicates().shape[0]

print("Количество уникальных строк:", unique_rows_count)

Количество уникальных строк: 9635


In [4]:
# Установка пакета SpaCy для русского языка
!python -m spacy download ru_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 52.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
# Загрузка модели SpaCy для русского языка
nlp = spacy.load("ru_core_news_sm")

# Функция для проверки, содержит ли текст символы, которые необходимы в дальнейшем анализе
def contains_code(token):
    return '#' in token or '+' in token

# Функция для лемматизации текста, которая работает со специфичным символьным output spacy
def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_tokens = []
    for token in doc:
        if contains_code(token.text):
            lemmatized_tokens.append(token.text)
        else:
            lemmatized_tokens.append(token.lemma_)

    lemmatized_text = ' '.join(lemmatized_tokens)

    lemmatized_text = lemmatized_text.replace(' #', '#').replace(' +', '+') \
                                       .replace(' /', '/').replace(' -', '-') \
                                       .replace(' @', '@').replace(' .', '.') \
                                       .replace(' <', '<').replace(' >', '>') \
                                       .replace(' {', '{').replace(' }', '}') \
                                       .replace(' [', '[').replace(' ]', ']') \
                                       .replace(' :', ':')

    lemmatized_text = re.sub(r'\s*,\s*', ',', lemmatized_text)

    return lemmatized_text

# Функция для обработки одной строки
def process_row(row):
    original_text = row.iloc[0]
    return lemmatize_text(original_text)

# Объект tqdm для отслеживания прогресса
with tqdm(total=len(data_for)) as progress_bar:
    # Используем ThreadPoolExecutor для многопоточной обработки
    with ThreadPoolExecutor(max_workers=20) as executor:
        futures = []
        for _, row in data_for.iterrows():
            future = executor.submit(process_row, row)
            futures.append(future)

        for index, future in enumerate(futures):
            lemmatized_text = future.result()
            data_for.at[index, 'lemmatized_text'] = lemmatized_text
            progress_bar.update(1)

print(data_for)


100%|██████████| 10489/10489 [03:08<00:00, 55.73it/s]

                                              raw_skills  \
0           figma, ui, английский язык, ux, android, ios   
1      английский язык, amocrm, телефония, программир...   
2      пользователь пк, adobe photoshop, грамотная ре...   
3      деловая коммуникация, грамотность, поиск инфор...   
4      linux, postgresql, sql, ms sql, java, oracle, ...   
...                                                  ...   
10484     кибербезопасность, информационная безопасность   
10485                                             асу тп   
10486  администрирование сетевого оборудования, настр...   
10487  linux, ci/cd pipeline, k8s, okd, ocp, менторст...   
10488  работа в команде, креативность, творческое мыш...   

                                         lemmatized_text  
0                figma,ui,английский язык,ux,android,ios  
1      английский язык,amocrm,телефония,программирова...  
2      пользователь пк,adobe photoshop,грамотный речь...  
3      деловой коммуникация,грамотность,пои

In [6]:
# Функция для токенизации текста с сохранением символов вместе со словами, с учетом особенностей
def tokenize_text(text):
    tokens = re.findall(r'\S+', text) # Разделение текста на токены по любому непробельному символу
    return tokens

def process_tokens(tokens):
    # Удаляем символы "'" из каждого токена
    tokens = [token for token in tokens]
    return tokens

# Применяем токенизацию к столбцу 'lemmatized_text'
data_for['tokens'] = data_for['lemmatized_text'].apply(tokenize_text)

# Применяем замену символов "'" на пустую строку после токенизации
data_for['tokens'] = data_for['tokens'].apply(process_tokens)

data_for

,raw_skills,lemmatized_text,tokens
0,"figma, ui, английский язык, ux, android, ios","figma,ui,английский язык,ux,android,ios","[figma,ui,английский, язык,ux,android,ios]"
1,"английский язык, amocrm, телефония, программир...","английский язык,amocrm,телефония,программирова...","[английский, язык,amocrm,телефония,программиро..."
2,"пользователь пк, adobe photoshop, грамотная ре...","пользователь пк,adobe photoshop,грамотный речь...","[пользователь, пк,adobe, photoshop,грамотный, ..."
3,"деловая коммуникация, грамотность, поиск инфор...","деловой коммуникация,грамотность,поиск информа...","[деловой, коммуникация,грамотность,поиск, инфо..."
4,"linux, postgresql, sql, ms sql, java, oracle, ...","linux,postgresql,sql,ms sql,java,oracle,субд,h...","[linux,postgresql,sql,ms, sql,java,oracle,субд..."
...,...,...,...
10484,"кибербезопасность, информационная безопасность","кибербезопасность,информационный безопасность","[кибербезопасность,информационный, безопасность]"
10485,асу тп,асу тп,"[асу, тп]"
10486,"администрирование сетевого оборудования, настр...","администрирование сетевой оборудование,настрой...","[администрирование, сетевой, оборудование,наст..."
10487,"linux, ci/cd pipeline, k8s, okd, ocp, менторст...","linux,ci/ cd pipeline,k8s,okd,ocp,менторство,н...","[linux,ci/, cd, pipeline,k8s,okd,ocp,менторств..."


In [7]:
# Функция для очистки токенов
def clean_tokens(tokens):
    cleaned_tokens = []
    for token in tokens:
        # Проверяем, является ли токен отдельно стоящим символом (например, /)
        if re.match(r'^[#\+/\-]+$', token):
            continue  # Пропускаем этот токен
        cleaned_tokens.append(token)

    return cleaned_tokens

# Применяем функцию к столбцу 'tokens' и сохраняем результат в новый столбец 'cleaned_tokens'
data_for['cleaned_tokens'] = data_for['tokens'].apply(clean_tokens)

data_for[['tokens', 'cleaned_tokens']]

,tokens,cleaned_tokens
0,"[figma,ui,английский, язык,ux,android,ios]","[figma,ui,английский, язык,ux,android,ios]"
1,"[английский, язык,amocrm,телефония,программиро...","[английский, язык,amocrm,телефония,программиро..."
2,"[пользователь, пк,adobe, photoshop,грамотный, ...","[пользователь, пк,adobe, photoshop,грамотный, ..."
3,"[деловой, коммуникация,грамотность,поиск, инфо...","[деловой, коммуникация,грамотность,поиск, инфо..."
4,"[linux,postgresql,sql,ms, sql,java,oracle,субд...","[linux,postgresql,sql,ms, sql,java,oracle,субд..."
...,...,...
10484,"[кибербезопасность,информационный, безопасность]","[кибербезопасность,информационный, безопасность]"
10485,"[асу, тп]","[асу, тп]"
10486,"[администрирование, сетевой, оборудование,наст...","[администрирование, сетевой, оборудование,наст..."
10487,"[linux,ci/, cd, pipeline,k8s,okd,ocp,менторств...","[linux,ci/, cd, pipeline,k8s,okd,ocp,менторств..."


In [8]:
nltk.download('stopwords')

def remove_stopwords(tokens):
    russian_stopwords = stopwords.words("russian")
    english_stopwords = stopwords.words("english")

    filtered_tokens = [word for word in tokens if word not in russian_stopwords and word not in english_stopwords]

    # Удаляем пробелы после запятых
    filtered_text = ', '.join(filtered_tokens)  # Объединяем токены в строку
    filtered_text = re.sub(r',\s+', ',', filtered_text)  # Убираем пробелы после запятых

    return filtered_text.split(',')  # Разбиваем строку обратно на токены

# Применяем обработку токенов для удаления стоп-слов
data_for['wo_stop'] = data_for['cleaned_tokens'].apply(remove_stopwords)

print(data_for)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


                                              raw_skills  \
0           figma, ui, английский язык, ux, android, ios   
1      английский язык, amocrm, телефония, программир...   
2      пользователь пк, adobe photoshop, грамотная ре...   
3      деловая коммуникация, грамотность, поиск инфор...   
4      linux, postgresql, sql, ms sql, java, oracle, ...   
...                                                  ...   
10484     кибербезопасность, информационная безопасность   
10485                                             асу тп   
10486  администрирование сетевого оборудования, настр...   
10487  linux, ci/cd pipeline, k8s, okd, ocp, менторст...   
10488  работа в команде, креативность, творческое мыш...   

                                         lemmatized_text  \
0                figma,ui,английский язык,ux,android,ios   
1      английский язык,amocrm,телефония,программирова...   
2      пользователь пк,adobe photoshop,грамотный речь...   
3      деловой коммуникация,грамотность

In [9]:
unique_count = data_for['wo_stop'].apply(lambda x: ', '.join(x)).nunique()

print(f"Уникальное число строк в 'wo_stop': {unique_count}")

Уникальное число строк в 'wo_stop': 9635


In [10]:
data_for['wo_stop'] = data_for['wo_stop'].apply(lambda x: ', '.join(x))

# Сохраняем DataFrame в файл Excel
data_for['wo_stop'].to_excel('/content/sample_data/ks_23.xlsx', index=False, sheet_name='Sheet1')
